In [1]:
import warnings
warnings.filterwarnings(action='ignore')
from functools import partial
from shapely import ops
import pyproj
from shapely.geometry import Point, Polygon
import numpy as np
import folium
import geopandas as gpd
import json, math
import matplotlib.pyplot as plt
import pandas as pd
import konlpy
import re
from geopy.distance import geodesic
from folium import plugins
import folium
import geopandas as gpd
from shapely.geometry import MultiPoint,Point
import os

In [2]:
os.chdir('../data')

In [3]:
os.getcwd()

'/Users/jominju/Desktop/대학교/contest/Car_Accident/최종/data'

In [4]:
def changeToPoint(x):
    lst = []
    for i in x:
        lst = lst + list(i.coords)
    return lst
def makePointDF(geometry_point_series):
    lst = geometry_point_series.sum()
    point_data = pd.DataFrame(lst,columns = ("lon", "lat"))
    point_data["Point"] = point_data.apply(lambda x: Point(x[0],x[1]),axis =1)
    return point_data

    
def draw_plot(point_data):
    fig,ax = plt.subplots(1,1,sharex=True,sharey=True,figsize=(20,20))
    road_data.plot(ax = ax, color = "blue",alpha = 0.2)
    point_data.plot.scatter(ax = ax, x="lon", y = "lat", s = 2, colormap='viridis',color = "red")

def draw_map(point_data):
    # map
    map1 = folium.Map(location=center, zoom_start=13, control_scale=True)

    # all subgroups
    all_subgroups = folium.FeatureGroup(name='all_thing')
    map1.add_child(all_subgroups)

    #osan
    osan_region = plugins.FeatureGroupSubGroup(all_subgroups, 'osan region')
    map1.add_child(osan_region)
    osan = gpd.read_file('../data/original/33.오산시_법정경계(읍면동).geojson',encoding="utf-8")
    folium.GeoJson(osan, style_function=lambda x:{'fillColor': 'transparent', 'color': 'grey'}).add_to(osan_region)

    road = plugins.FeatureGroupSubGroup(all_subgroups, 'road')
    map1.add_child(road)
    folium.GeoJson(road_data, style_function=lambda x:{'fillColor': 'transparent', "weight": int(x["properties"]["width"]) ** 1.5 / 2, 'color': 'black'}).add_to(road)

    # point
    check_point = plugins.FeatureGroupSubGroup(all_subgroups, 'check_point')
    map1.add_child(check_point)
    for i in range(0, len(point_data)):
        folium.Circle(
            location = [point_data.loc[i,"lat"],
                        point_data.loc[i,"lon"]],
            radius = 1,
            color = "red",
            fillColor = "red",
            tooltip=str((point_data.loc[i,"lon"], point_data.loc[i,"lat"])),
        ).add_to(check_point)

    folium.LayerControl(collapsed=False).add_to(map1)
    return map1

In [5]:
center = (37.1623799231016, 127.05436890115905)
road_data = gpd.read_file('23.오산시_상세도로망_LV6.geojson',encoding="utf-8")
road_data = road_data[road_data["road_name"].apply(lambda x: "고속도로" not in str(x))].reset_index(drop = True)
road_data["geometry_point"] = road_data["geometry"].apply(changeToPoint)

In [6]:
def refine_point_data(point_data, stan_range):
    point_data = point_data.copy()
    index = 0
    while index < len(point_data):
        a,b,c = point_data.iloc[index]
        tmp_dist = point_data.apply(lambda x : geodesic((b, a),(x[1], x[0])).m ,axis = 1)
        point_data = point_data[(tmp_dist <= 0) | (tmp_dist  >= stan_range)]
        index += 1
#         print(((tmp_dist <= 0) | (tmp_dist  >= stan_range)).sum())
        print(tmp_dist[~((tmp_dist <= 0) | (tmp_dist  >= stan_range))])
    return point_data

### 평균 간격 확인

In [7]:
def print_dist_info(geometry_point_series):
    def cal_dist(x):
        lst = []
        for i, j in list(zip(x[:-1],x[1:])):
            lst.append(geodesic((i[1], i[0]),(j[1], j[0])).m)
        return lst
    lst = geometry_point_series.apply(cal_dist).sum()
    print("개수", len(lst))
    print("평균", sum(lst)/ len(lst))
    print("최소", min(lst))
    print("최대", max(lst))
print_dist_info(road_data["geometry_point"])

개수 5334
평균 37.42586223979561
최소 1.2884774203999179
최대 442.0867340433362


### 1m ~ 2m 간격 데이터 형성

In [8]:
def split_point(x):
    result=[x[0]]
    for next_point in x[1:]:
        before_point = result[-1]
        dist = geodesic((before_point[1], before_point[0]),(next_point[1], next_point[0])).m
        if  dist >=2:
            num = int(dist // 1)
            x_step_size = (next_point[0] - before_point[0]) / num
            y_step_size = (next_point[1] - before_point[1]) / num
            for i in range(1,num):
                result.append((before_point[0] + x_step_size * i, before_point[1] + y_step_size * i))
        result.append(next_point)
    return result

In [9]:
tmp = road_data["geometry_point"].apply(split_point)
print_dist_info(tmp)

개수 196953
평균 1.0135897863332515
최소 1.0000002763786253
최대 1.9985362736327137


In [10]:
# 10개 간격
tmp = road_data["geometry_point"].apply(split_point).apply(lambda x: x[::20])
print_dist_info(tmp)

개수 8998
평균 20.20195120313476
최소 14.271985864082676
최대 22.86653494041831


### 생성된 데이터 간격별로 저장

In [12]:
tmp = road_data["geometry_point"].apply(split_point).apply(lambda x: x[::25])
print_dist_info(tmp)
tmp_point = makePointDF(tmp)
tmp_point[["lon","lat"]].to_csv("learn_point25.csv",index=False)

개수 7044
평균 25.218295879342072
최소 15.62639813709922
최대 27.83348623899768


In [13]:
tmp = road_data["geometry_point"].apply(split_point).apply(lambda x: x[::50])
print_dist_info(tmp)
tmp_point = makePointDF(tmp)
tmp_point[["lon","lat"]].to_csv("learn_point50.csv",index=False)

개수 3035
평균 50.17136865919107
최소 32.23569609389694
최대 53.640569839621755
